In [ ]:
from tictactoe.tictactoe import TicTacToe


caro_board = TicTacToe()
board = caro_board.initial_state
board, won = caro_board.move(board, 0, 0)
print("caro_board.convert_mcts_state_to_list_state(board)", caro_board.convert_mcts_state_to_list_state(board), won)
board, won = caro_board.move(board, 1, 1)
print("caro_board.convert_mcts_state_to_list_state(board)", caro_board.convert_mcts_state_to_list_state(board), won)
board, won = caro_board.move(board, 3, 0)
print("caro_board.convert_mcts_state_to_list_state(board)", caro_board.convert_mcts_state_to_list_state(board), won)
board, won = caro_board.move(board, 5, 1)
print("caro_board.convert_mcts_state_to_list_state(board)", caro_board.convert_mcts_state_to_list_state(board), won)
board, won = caro_board.move(board, 6, 0)
print("caro_board.convert_mcts_state_to_list_state(board)", caro_board.convert_mcts_state_to_list_state(board), won)

In [ ]:
import numpy as np
import torch
from utils import cls, draw_matrix
from model import Net
from mcts.mcts import MCTS
from tictactoe.tictactoe import TicTacToe
import config as cfg

_size = 3
game = TicTacToe(n=_size)
mcts_store = MCTS(game)
state = game.initial_state

model = Net(input_shape=game.obs_shape, actions_n=game.action_space)
model.load_state_dict(torch.load(
            'trained/trained_tictactoe/best_005_00900.dat', map_location=lambda storage, loc: storage))

won = False
valid_moves = _size * _size

while not won or valid_moves > 0:
  row = input("Enter row: ")
  column = input("Enter column: ")
  cls()
  _move = int(row) * _size + int(column)
  state, won = game.move(state, _move, 0)
  draw_matrix(game.convert_mcts_state_to_list_state(state))
  if won:
    print("You won!")
  valid_moves = len(game.possible_moves(state))
  if valid_moves == 0:
    break

  mcts_store.search_batch(
              cfg.BOT_MCTS_SEARCHES, cfg.BOT_MCTS_BATCH_SIZE, state, 1, model)
  probs, values = mcts_store.get_policy_value(state, tau=0)
  action = np.random.choice(game.action_space, p=probs)
  state, won = game.move(state, action, 1)
  draw_matrix(game.convert_mcts_state_to_list_state(state))
  if won:
    print("You lose!!!")
  valid_moves = len(game.possible_moves(state))
if valid_moves == 0:
  print("It's a draw")
